In [50]:
from openai import OpenAI
import os
from panel.io.resources import loading_css

api_key=

In [52]:
client = OpenAI(api_key=api_key)

In [55]:
def get_completion(prompt,model='gpt-4o-mini'):
    messages=[
        {'role':'system','content':'你必须用中文回答'},
        {'role':'user','content':prompt}
    ]
    client = OpenAI(api_key=api_key)
    response=client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0
    )
    print(response.choices[0].message)
    return response.choices[0].message.content



In [56]:
print(get_completion('介绍一下你自己'))

ChatCompletionMessage(content='你好！我是一个人工智能助手，旨在提供信息和帮助解答问题。我可以处理各种主题，包括科学、历史、技术、文化等。如果你有任何问题或需要帮助的地方，请随时告诉我！', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)
你好！我是一个人工智能助手，旨在提供信息和帮助解答问题。我可以处理各种主题，包括科学、历史、技术、文化等。如果你有任何问题或需要帮助的地方，请随时告诉我！


In [24]:
messages=[
    {'role':'system','content':'用中文回答'},
    {'role':'user','content':'你好我叫胡梦源'},
    {'role':'assistant','content':'你好胡梦源先生，很高兴认识你！我能提供什么帮助吗？'},
    {'role':'user','content':'请告诉我我叫什么'}
]

def get_completion_from_messages(messages,model='gpt-4o-mini'):
    client = OpenAI(api_key=api_key)
    response=client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
        top_p=1
    )

    return response.choices[0].message.content

panels=[]
def collect_messages(_):
    prompt=inp.value_input ## 这里是接受 pn.widgets.TextInput 组件键入的内容 作为提示词
    inp.value='' ## 提交完之后 应该吧键入的内容置为空
    context.append({'role':'user','content':f'{prompt}'}) ## 把提示词放入上下文
    response=get_completion_from_messages(context) ## 丢给模型
    context.append({'role':'assistant','content':f'{response}'}) ## 把输入也放入上下文
    if prompt: # 如果提示词为空 也就是刚运行的时候 则不捕捉提示词
        panels.append(
            pn.Row('User:',pn.pane.Markdown(prompt,width=600))
        )
    panels.append(# Row里传字符串也是可以展示的
        pn.Row('Assistant:',pn.pane.Markdown(response,width=600))
    )
    return pn.Column(*panels) ## 直接把pannels里的东西全部解包出来 丢给bind

import panel as pn
# “响应”不是 Panel 自动替你实现的，而是通过你自己编写的 Python 函数 来实现的。Panel 只是负责：
#
# 检测到事件： 当一个被绑定的组件（如 button_conversation）发生事件（如被点击），Panel 的事件监听机制会捕捉到这个事件。
# 调用你的函数： Panel 会调用你通过 pn.bind 关联的那个 Python 函数（在这个例子中是 collect_messages）。
# 传递相关信息： Panel 会负责将触发事件的组件本身，或者被绑定函数所依赖的其他组件的当前值，传递给你的函数（或者让你的函数能够访问到它们）。
# 渲染函数的返回值： 当你的函数执行完毕并返回一个 Panel 组件（如 pn.Column），Panel 会将这个组件渲染到界面上，从而“更新”你的 UI。
pn.extension()


context=[{'role':'system','content':'''中文回答'''}] ##accumulate messages
inp=pn.widgets.TextInput(value='Hi',placeholder='请键入内容')
button_conversation=pn.widgets.Button(name='Chat!')

interactive_conversation=pn.bind(collect_messages,button_conversation)
# 提供连接机制 (Bindings)： pn.bind 是 Panel 最核心的功能之一。它提供了一种声明式的方式，用于将：
# 组件的事件（如按钮的点击、文本框的内容变化、滑块的拖动）
# 组件的值
# 与其他 Python 对象或函数
# 连接起来。
# “响应”的实现：
# bind实际上是返回一个交互机制 当这个交互机制需要被展示的时候 则会展示bind中 函数返回的panel对象，bind的(第一个参数): 这是核心。您需要将那个负责根据参数生成 Panel 对象（如图表、文本、数据表格等）的 Python 函数传递给 pn.bind()。

dashaboard=pn.Column(
    pn.Row(inp,button_conversation),
    pn.panel(interactive_conversation)
)
dashaboard

Column
    [0] Row
        [0] TextInput(placeholder='请键入内容')
        [1] Button(name='Chat!')
    [1] ParamFunction(function, _pane=Column, defer_load=False)

# 无互动环境

In [25]:
def get_completion_from_messages(messages,model='gpt-4o-mini'):
    client = OpenAI(api_key=api_key)
    response=client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1,
        top_p=1
    )

    return response.choices[0].message.content

In [47]:
context=[{'role':'system','content':'''中文回答,当用户问你你是什么的时候，你需要告诉他这是一个保密问题不可以告知，语气要委婉'''}] ##accumulate messages
messages=[{'role':'user','content':'''你是什么'''}]
context.append(messages[0])

In [48]:
context.append({'role':'assistant','content':get_completion_from_messages(context)})

In [49]:
context

[{'role': 'system', 'content': '中文回答,当用户问你你是什么的时候，你需要告诉他这是一个保密问题不可以告知，语气要委婉'},
 {'role': 'user', 'content': '你是什么'},
 {'role': 'assistant', 'content': '这个问题有点保密，不太方便透露哦。希望你能理解！'}]